In [1]:
!python example_flow.py run --n 9
!python upload_folder_flow.py run --dir .

Metaflow 2.7.23 executing ExampleFlow for user:jovyan
Validating your flow...
    The graph looks good!
Running pylint...
    Pylint is happy!
2023-07-17 22:41:17.498 Workflow starting (run-id 1689633677373213):
2023-07-17 22:41:17.510 [1689633677373213/start/1 (pid 2252)] Task is starting.
2023-07-17 22:41:19.508 [1689633677373213/start/1 (pid 2252)] Task finished successfully.
2023-07-17 22:41:19.557 [1689633677373213/save_table/2 (pid 2282)] Task is starting.
2023-07-17 22:41:23.775 [1689633677373213/save_table/2 (pid 2282)] Task finished successfully.
2023-07-17 22:41:23.821 [1689633677373213/end/3 (pid 2329)] Task is starting.
2023-07-17 22:41:25.126 [1689633677373213/end/3 (pid 2329)] All done!
2023-07-17 22:41:27.167 [1689633677373213/end/3 (pid 2329)] Task finished successfully.
2023-07-17 22:41:27.192 Done!
Metaflow 2.7.23 executing UploadFlow for user:jovyan
Validating your flow...
    The graph looks good!
Running pylint...
    Pylint is happy!
2023-07-17 22:41:33.569 Workfl

In [2]:
import metaflow
import pandas as pd
import opal.flow
    
s3 = opal.flow.minio_s3fs()

ex_flow = metaflow.Flow("ExampleFlow")
up_flow = metaflow.Flow("UploadFlow")
ex_run = ex_flow.latest_successful_run
up_run = up_flow.latest_successful_run
    
pd.read_parquet(ex_run.data.data_files['parquet_table'], filesystem=s3)

prod
0_x 0_y      
1   1       1
    2       2
    3       3
    4       4
    5       5
...       ...
8   4      32
    5      40
    6      48
    7      56
    8      64

[64 rows x 1 columns]

In [4]:
up_run.tags

frozenset({'metaflow_version:2.7.23',
           'python_version:3.10.9',
           'runtime:dev',
           'user:jovyan'})